In [1]:
# Gerekli Kütüphaneleri Yükleyin
import pandas as pd
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Özellikleri Ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA ve LDA Dönüşümleri
pca = PCA(n_components=10)
lda = LDA()

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

# Algoritmalar ve İsimleri
models = {
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=200),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000)
}

# Sonuçları Saklamak İçin Liste
time_results = []

# PCA, LDA ve No Reduction Durumlarını İçeren Modellerin Eğitimi ve Test Sürelerinin Hesaplanması
reductions = {
    "No Reduction": (X_train_scaled, X_test_scaled),
    "With PCA": (X_train_pca, X_test_pca),
    "With LDA": (X_train_lda, X_test_lda)
}

# Normal Eğitim ve Test Süreleri
for reduction_name, (X_tr, X_te) in reductions.items():
    for model_name, model in models.items():
        # Eğitim Süresi
        start_train = time.time()
        model.fit(X_tr, y_train)
        end_train = time.time()
        train_time = end_train - start_train

        # Test Süresi
        start_test = time.time()
        model.predict(X_te)
        end_test = time.time()
        test_time = end_test - start_test

        # Sonuçları Listeye Ekleyin
        time_results.append({
            "Reduction": reduction_name,
            "Model": model_name,
            "Train Time (seconds)": train_time,
            "Test Time (seconds)": test_time
        })

# K-Fold Çapraz Doğrulama Sürelerinin Hesaplanması
kf = KFold(n_splits=3)
for reduction_name, (X_tr, X_te) in reductions.items():
    for model_name, model in models.items():
        fold_train_time = 0
        fold_test_time = 0
        for train_index, test_index in kf.split(X_tr):
            X_fold_train, X_fold_test = X_tr[train_index], X_tr[test_index]
            y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]

            # Eğitim Süresi
            start_train = time.time()
            model.fit(X_fold_train, y_fold_train)
            end_train = time.time()
            fold_train_time += end_train - start_train

            # Test Süresi
            start_test = time.time()
            model.predict(X_fold_test)
            end_test = time.time()
            fold_test_time += end_test - start_test

        # Ortalama Süreleri Hesaplama
        fold_train_time /= 3
        fold_test_time /= 3

        # Sonuçları Listeye Ekleyin
        time_results.append({
            "Reduction": f"{reduction_name} (K-Fold)",
            "Model": model_name,
            "Train Time (seconds)": fold_train_time,
            "Test Time (seconds)": fold_test_time
        })

# Sonuçları DataFrame Olarak Gösterme
time_df = pd.DataFrame(time_results)

# Performans Sonuçları Tablosunu Görüntüleme
print("Eğitim ve Test Süreleri Tablosu (Reduction):")
print(time_df)

C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:29:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:40:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:45:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\

Eğitim ve Test Süreleri Tablosu (Reduction):
                Reduction                Model  Train Time (seconds)  \
0            No Reduction        Random Forest             31.181422   
1            No Reduction              XGBoost              0.220304   
2            No Reduction                  SVM            625.330569   
3            No Reduction                 k-NN              0.006839   
4            No Reduction  Logistic Regression              0.116645   
5                With PCA        Random Forest             36.648140   
6                With PCA              XGBoost              0.170847   
7                With PCA                  SVM            255.159445   
8                With PCA                 k-NN              0.053511   
9                With PCA  Logistic Regression              0.038008   
10               With LDA        Random Forest             20.611531   
11               With LDA              XGBoost              0.069319   
12               Wi